## 2024 07/30 AutoML - Hawkeye Hands

*Last Updated*: 2024-07-14

### Authors
* Nicole Tin


### Overview
This Jupyter notebook is intended to demonstrate ...


### Key Results

- ...

In [1]:
# --- Imports
import shap
import pandas as pd

from pycaret import regression

# import mlflow # creates error


In [2]:
# Dataset
dataset_name = "hawkeye-hands"

# AutoML
experiment_name = "NT_hawkeye-hands-texture-paralleled"
num_best_models = 5
random_seed = 42

# Paths
root = '/Users/nicole/Documents/DermaML_local/hawkeye-hands-2024-07-29'
# src_dir = '/Users/nicole/Documents/DermaML_local/hawkeye-hands-2024-07-29/processed_images/'
features_dir =  '/Users/nicole/Documents/DermaML_local/hawkeye-hands-2024-07-29/features-2024-08-02/'

image_folder = '/processed_images/'
csv_file = '/metadata.csv'

In [3]:
# -- Read the CSV file
metadata = pd.read_csv(root+ csv_file)
metadata.loc[:, 'Age'] = 2024-metadata['birth_year']
valid_image_fnames_df = pd.DataFrame(metadata.set_index('Age').loc[:, ['right_hand_image_file', 'left_hand_image_file']].stack()).reset_index()
valid_image_fnames_df.columns = ['Age', 'handedness', 'filename']
valid_image_fnames_df.loc[:,'filename'] = valid_image_fnames_df['filename'].apply(lambda x: x[:-5])
valid_image_fnames = valid_image_fnames_df['filename'].to_numpy()

In [4]:
f_dermaml = pd.read_csv(features_dir+'2024-08-02_hawkeye-hands_dermaml-features.csv').drop(columns='Unnamed: 0')
f_pyfeats = pd.read_csv(features_dir+'2024-08-02_hawkeye-hands_pyfeats_textures.csv').drop(columns='Unnamed: 0')
f_redness = pd.read_csv(features_dir+'2024-08-02_hawkeye-hands-redness-features.csv').drop(columns='Unnamed: 0')
f_hessian = pd.read_csv(features_dir+'2024-08-02_hawkeye-hands-hessian-features.csv').drop(columns='Unnamed: 0')

F = f_redness.merge(f_hessian).merge(f_pyfeats).merge(f_dermaml)
F.loc[:,'filename'] = F['filename'].apply(lambda x:x[:-4])
X = F.join(valid_image_fnames_df.set_index('filename'), on='filename', how='inner').drop(columns=['handedness', 'filename'])

In [18]:
# X.to_csv('2024-08-01_NT_Hawkeye-Hands-Texture-Features.csv')
X['Age'].describe()

count    576.000000
mean      46.715278
std       49.094785
min        2.000000
25%       24.000000
50%       37.500000
75%       62.250000
max      789.000000
Name: Age, dtype: float64

In [20]:
metadata.columns

Index(['record_id', 'gender', 'gender_specify', 'birth_year',
       'sex_assigned_at_birth', 'sex_assigned_at_birth_specify',
       'race_ethnicity', 'race_ethnicity_specify', 'ethnicity', 'handedness',
       'occupation', 'driving_time', 'sun_exposure', 'sunscreen_use', 'region',
       'region_specify', 'left_hand_image_file', 'right_hand_image_file',
       'form_complete', 'Age'],
      dtype='object')

In [19]:
X.loc[X['Age'] > 90]

,relative_redness_mean,relative_redness_std,GLCM_ASM_Mean_wrinkles_pyfeats,GLCM_Contrast_Mean_wrinkles_pyfeats,GLCM_Correlation_Mean_wrinkles_pyfeats,GLCM_SumOfSquaresVariance_Mean_wrinkles_pyfeats,GLCM_InverseDifferenceMoment_Mean_wrinkles_pyfeats,GLCM_SumAverage_Mean_wrinkles_pyfeats,GLCM_SumVariance_Mean_wrinkles_pyfeats,GLCM_SumEntropy_Mean_wrinkles_pyfeats,...,lbp_6,lbp_7,lbp_8,lbp_9,lbp_10,contrast_scikit,correlation_scikit,energy_scikit,homogeneity_scikit,Age
12,45.761472,74.251101,0.005053,44.179798,0.992374,2919.330394,0.515174,75.757733,11633.141780,7.010151,...,0.026459,0.019170,0.019039,0.722254,0.054027,852.910688,0.665251,0.684187,0.771479,100
23,41.745221,73.108585,0.003979,39.400977,0.992857,2772.724436,0.488773,84.952849,11051.496767,7.377674,...,0.024249,0.017422,0.017461,0.750040,0.049403,215.609972,0.909477,0.718258,0.799049,94
26,72.279300,89.806733,0.002785,51.189502,0.984211,1637.536353,0.492339,102.058405,6498.955910,7.359470,...,0.037424,0.026578,0.027037,0.599443,0.080446,965.298797,0.663160,0.557946,0.666116,97
48,80.596506,93.017228,0.002590,51.768242,0.976571,1110.045211,0.465327,112.933012,4388.412603,7.247332,...,0.038045,0.028346,0.031433,0.566954,0.097762,801.308218,0.713006,0.525973,0.645630,97
58,57.095766,83.264957,0.007487,37.550273,0.994430,3398.627792,0.565898,74.528264,13556.960895,6.865657,...,0.030406,0.023078,0.023095,0.683914,0.062233,1259.991551,0.618019,0.628432,0.739906,95
60,73.113858,87.961526,0.005240,35.959542,0.992546,2434.794660,0.564429,77.070232,9703.219097,7.054377,...,0.039580,0.030017,0.028905,0.603177,0.076669,1266.642169,0.596163,0.538204,0.677351,93
65,68.742687,84.258527,0.002216,61.817465,0.983056,1839.210576,0.446278,101.550705,7295.024837,7.547696,...,0.036074,0.027750,0.030235,0.590488,0.090447,1009.333795,0.673358,0.543970,0.648858,91
95,79.869970,92.516396,0.002423,43.743032,0.974918,880.335014,0.435092,129.837847,3477.597025,7.173342,...,0.034356,0.028401,0.034781,0.569102,0.112920,841.644743,0.726070,0.524774,0.642992,91
98,46.085701,71.439711,0.004282,72.913873,0.991241,4171.648725,0.426064,115.862332,16613.681028,7.413607,...,0.027909,0.021835,0.024299,0.680092,0.072098,892.268286,0.805370,0.643743,0.720804,789
203,71.138928,82.589261,0.003773,45.152251,0.989009,2072.240195,0.493109,75.091198,8243.808527,7.216904,...,0.040414,0.029861,0.029224,0.577232,0.081449,1196.814056,0.579495,0.518865,0.645567,91


1. engineered features only
2. + GLCM wrinkles
3. CNN

preliminary ML model; a lot more interesting things to do be done with features (collaboration) and model improvement;
- describe features
- share model results
- interpretation of graph

In [22]:
# --- Perform AutoML Evaluation

# Set up the dataset for AutoML regression
regression.setup(data=X,
                 target="Age",
                 log_experiment=True,
                 experiment_name=experiment_name,
                 session_id=random_seed,
                ) 

best_models = regression.compare_models(n_select=num_best_models, verbose=False)


,Description,Value
0,Session id,42
1,Target,Age
2,Target type,Regression
3,Original data shape,"(576, 50)"
4,Transformed data shape,"(576, 50)"
5,Transformed train set shape,"(403, 50)"
6,Transformed test set shape,"(173, 50)"
7,Numeric features,49
8,Preprocess,True
9,Imputation type,simple


In [23]:
regression.pull()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,1.066290e+01,1.454859e+03,2.234380e+01,6.404000e-01,0.3257,3.065000e-01,0.283
lightgbm,Light Gradient Boosting Machine,1.161990e+01,1.503886e+03,2.445290e+01,5.231000e-01,0.3677,3.507000e-01,0.264
ada,AdaBoost Regressor,1.418360e+01,1.508368e+03,2.496850e+01,4.922000e-01,0.4046,4.452000e-01,0.111
dt,Decision Tree Regressor,1.252840e+01,1.540850e+03,2.614060e+01,4.136000e-01,0.3909,2.958000e-01,0.024
et,Extra Trees Regressor,1.202820e+01,1.589911e+03,2.629800e+01,3.629000e-01,0.3407,3.081000e-01,0.207
rf,Random Forest Regressor,1.194990e+01,1.618763e+03,2.654320e+01,3.096000e-01,0.3512,3.390000e-01,0.511
ridge,Ridge Regression,1.686670e+01,1.740867e+03,2.925480e+01,1.846000e-01,0.5158,4.946000e-01,0.018
en,Elastic Net,1.718180e+01,1.732129e+03,2.943440e+01,1.670000e-01,0.4945,5.013000e-01,0.022
lasso,Lasso Regression,1.715700e+01,1.741041e+03,2.949670e+01,1.647000e-01,0.4935,5.001000e-01,0.020
omp,Orthogonal Matching Pursuit,1.862180e+01,1.736762e+03,2.980230e+01,1.587000e-01,0.5369,5.706000e-01,0.018


In [16]:
dt = regression.create_model('dt')
regression.tune_model(dt, optimize='RMSE')
# final_model = regression.finalize_model(best_models[3])
regression.save_experiment('hawkeye-hands-texture-fraction')

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,12:51:59
Status,. . . . . . . . . . . . . . . . . .,Selecting Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Decision Tree Regressor


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,11.7642,282.9133,16.8200,0.4073,0.3827,0.2731
1,9.8537,252.0000,15.8745,0.3907,0.3596,0.2677
2,11.3659,242.5854,15.5752,0.5664,0.4728,0.5513
3,12.0250,335.5750,18.3187,0.3867,0.4189,0.3507
4,9.6000,193.1000,13.8960,0.5448,0.3160,0.2409
5,11.0750,317.8250,17.8276,0.4027,0.3673,0.2545
6,12.7500,322.4000,17.9555,0.5430,0.3877,0.3257
7,26.4500,12974.2000,113.9043,0.0717,0.4819,0.2269
8,9.4250,251.9250,15.8721,0.3676,0.3697,0.2040


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,14.0000,345.4146,18.5853,0.2763,0.4210,0.3581
1,9.5366,172.5976,13.1376,0.5827,0.4845,0.6505
2,13.0488,354.9634,18.8405,0.3655,0.5312,0.6512
3,15.3375,375.5062,19.3780,0.3137,0.4369,0.4149
4,11.9500,253.1500,15.9107,0.4033,0.3410,0.2852
5,13.4625,386.3562,19.6559,0.2739,0.4739,0.3667
6,16.4500,492.6250,22.1952,0.3018,0.4304,0.3608
7,29.8500,13120.4250,114.5444,0.0612,0.5264,0.3140
8,11.1750,306.9250,17.5193,0.2296,0.3810,0.2582


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [10]:
tuned_models = [regression.tune_model(model, optimize='RMSE') for model in best_models]
ensem_models = [regression.ensemble_model(model, n_estimators=5, optimize='RMSE') for model in tuned_models]
tuned_blend = regression.blend_models(tuned_models, optimize='RMSE')
ensem_blend = regression.blend_models(ensem_models, optimize='RMSE')
model = regression.automl(optimize='RMSE')

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,13:56:26
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Gradient Boosting Regressor


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,11.6002,199.3616,14.1195,0.5823,0.3290,0.3181
1,11.4445,197.9881,14.0708,0.5213,0.5357,0.8704
2,12.7113,236.8249,15.3891,0.5767,0.5064,0.7307
3,13.3204,246.8232,15.7106,0.5489,0.3771,0.3870
4,12.5128,217.4035,14.7446,0.4875,0.3417,0.3509
5,12.2346,208.2815,14.4320,0.6086,0.3530,0.3601
6,15.6669,346.4983,18.6145,0.5089,0.3836,0.3793
7,29.0794,13468.1960,116.0526,0.0363,0.5090,0.3333
8,11.9578,225.1329,15.0044,0.4349,0.3387,0.3320


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,9.6350,153.9060,12.4059,0.6776,0.2658,0.2408
1,10.2800,210.1879,14.4979,0.4918,0.5238,0.8214
2,12.8888,276.1406,16.6175,0.5064,0.5225,0.7876
3,14.7626,330.8817,18.1902,0.3953,0.4285,0.4494
4,11.5316,220.2410,14.8405,0.4809,0.3434,0.3295
5,12.9629,234.9582,15.3283,0.5584,0.3430,0.3484
6,15.4972,337.5315,18.3720,0.5216,0.3721,0.3732
7,29.0029,13055.5465,114.2609,0.0658,0.5074,0.3415
8,10.5463,176.8594,13.2989,0.5561,0.3028,0.2720


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


...

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,10.8572,188.3800,13.7252,0.6053,0.2960,0.2629
1,11.0995,198.6677,14.0950,0.5196,0.5214,0.8186
2,12.6564,258.7974,16.0872,0.5374,0.5311,0.8170
3,15.3627,734.9270,27.1095,-0.3431,0.4201,0.4219
4,10.3679,171.8774,13.1102,0.5949,0.3070,0.2819
5,11.0254,180.3085,13.4279,0.6612,0.3108,0.2913
6,15.4302,370.5206,19.2489,0.4748,0.3909,0.3657
7,27.5904,13082.1373,114.3772,0.0639,0.4683,0.2713
8,9.8584,175.0980,13.2325,0.5605,0.2953,0.2449


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,9.4285,176.2456,13.2758,0.6308,0.2865,0.2207
1,9.4098,186.0556,13.6402,0.5501,0.5316,0.9349
2,21.3805,5243.1795,72.4098,-8.3717,0.6770,1.2167
3,11.8633,256.5944,16.0186,0.5311,0.3731,0.3445
4,9.7950,164.2650,12.8166,0.6128,0.2866,0.2499
5,10.1500,197.1200,14.0399,0.6296,0.3053,0.2501
6,13.2800,344.8520,18.5702,0.5112,0.3959,0.3437
7,26.7550,12946.8990,113.7844,0.0736,0.4551,0.2409
8,8.9883,174.6748,13.2165,0.5615,0.2779,0.1989


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,10.1013,158.9512,12.6076,0.6670,0.2788,0.2563
1,10.9960,175.5581,13.2498,0.5755,0.4991,0.7495
2,12.5655,259.1648,16.0986,0.5368,0.5159,0.7499
3,13.1508,260.9116,16.1528,0.5232,0.3753,0.3729
4,12.0967,235.0407,15.3310,0.4460,0.3383,0.3347
5,13.6580,242.9683,15.5874,0.5434,0.3487,0.3611
6,16.7045,377.3574,19.4257,0.4651,0.3731,0.3789
7,28.9132,13254.5373,115.1284,0.0516,0.5014,0.3311
8,12.4494,285.6151,16.9002,0.2831,0.3521,0.3324


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,8.7459,134.5365,11.5990,0.7181,0.2529,0.2094
1,8.5637,128.5179,11.3366,0.6892,0.4546,0.6039
2,9.8403,172.3971,13.1300,0.6919,0.4653,0.6253
3,10.7655,208.8663,14.4522,0.6183,0.3515,0.3226
4,8.1366,108.9887,10.4398,0.7431,0.2456,0.2197
5,9.9341,161.6173,12.7129,0.6963,0.2826,0.2513
6,12.3445,237.8803,15.4234,0.6628,0.3280,0.2980
7,25.6389,13105.6812,114.4800,0.0622,0.4440,0.2165
8,7.8986,133.8271,11.5684,0.6641,0.2514,0.1914


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,8.4006,130.0488,11.4039,0.7275,0.2479,0.2044
1,9.2955,160.1091,12.6534,0.6129,0.5052,0.8188
2,15.1417,1040.7575,32.2608,-0.8603,0.5828,0.8984
3,11.6281,222.3238,14.9105,0.5937,0.3517,0.3364
4,10.0187,165.6069,12.8688,0.6096,0.2855,0.2635
5,10.9272,191.9876,13.8560,0.6392,0.3178,0.2957
6,13.9178,292.3867,17.0993,0.5856,0.3535,0.3318
7,26.1157,13032.5053,114.1600,0.0675,0.4469,0.2380
8,9.1321,170.3837,13.0531,0.5723,0.2781,0.2243


In [15]:
model

GradientBoostingRegressor(random_state=42)

In [12]:
final_model = regression.finalize_model(model)
regression.save_model(final_model, 'hawkeye-hands-texture-fraction')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['relative_redness_mean',
                                              'relative_redness_std',
                                              'GLCM_ASM_Mean_wrinkles_pyfeats',
                                              'GLCM_Contrast_Mean_wrinkles_pyfeats',
                                              'GLCM_Correlation_Mean_wrinkles_pyfeats',
                                              'GLCM_SumOfSquaresVariance_Mean_wrinkles_pyfeats',
                                              'GLCM_InverseDifferenceMoment_Mean_wrinkles_pyf...
                                              'NGTDM_Coarseness_ngtdm',
                                              'NGTDM_Contrast_ngtdm',
                                              'NGTDM_Busyness_ngtdm',
                                              'NGTDM_Complexity_ngtdm',
                                              'NG

In [ ]:
saved_model = regression.load_model('hawkeye-hands-texture-fraction')